# Leverage and Kelly Criterion Analysis

This notebook explores leverage strategies and Kelly criterion using historical S&P 500 data.

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import importlib
importlib.reload(load_data)

from load_data import load_sp500, load_fedfunds, load_all_data, adjust_returns_for_dividends, calculate_leveraged_returns

## Load Data

In [13]:
# Load both datasets
sp500, fedfunds = load_all_data()

print(f"S&P 500: {len(sp500)} rows from {sp500['Date'].min().date()} to {sp500['Date'].max().date()}")
print(f"Fed Funds: {len(fedfunds)} rows from {fedfunds['Date'].min().date()} to {fedfunds['Date'].max().date()}")

S&P 500: 9999 rows from 1979-12-26 to 2019-08-15
Fed Funds: 857 rows from 1954-07-01 to 2025-11-01


## Explore S&P 500 Data

In [14]:
sp500[['Date', 'Price', 'Raw Change (bps)', 'Days Since Prev']].head(10)

,Date,Price,Raw Change (bps),Days Since Prev
0,1979-12-26,107.8,9.0,0
1,1979-12-27,108.0,19.0,1
2,1979-12-28,107.8,-19.0,1
3,1979-12-31,107.9,9.0,3
4,1980-01-01,107.9,0.0,1
5,1980-01-02,105.8,-195.0,1
6,1980-01-03,105.2,-57.0,1
7,1980-01-04,106.5,124.0,1
8,1980-01-07,106.8,28.0,3
9,1980-01-08,108.9,197.0,1


In [15]:
sp500[['Price', 'Change %']].describe()

,Price,Change %
count,9999.000000,9999.000000
mean,979.048775,0.000388
std,720.915010,0.010960
min,98.200000,-0.204800
25%,328.350000,-0.004500
50%,974.100000,0.000500
75%,1359.550000,0.005600
max,3025.900000,0.115900


## Your Analysis Here

Add your leverage and Kelly criterion calculations below...

# Simple example with constant 2% dividend yield
adjusted_simple = adjust_returns_for_dividends(sp500, annual_div_yield=0.02)

print("Original vs Total Returns:")
print(f"Raw avg daily return: {sp500['Raw Change (bps)'].mean():.2f} bps")
print(f"Total avg daily return: {adjusted_simple['Total Return (bps)'].mean():.2f} bps")
print(f"Dividend contribution: {adjusted_simple['Daily Div Yield (bps)'].mean():.2f} bps")
print(f"Expense drag: {adjusted_simple['Expense (bps)'].mean():.2f} bps")

adjusted_simple[['Date', 'Days Since Prev', 'Raw Change (bps)', 'Daily Div Yield (bps)', 'Total Return (bps)']].head(10)

In [16]:
# Time-varying dividend yields based on historical estimates (from README)
div_schedule = {
    (1975, 1982): 0.045,  # 4.5% mid-70s to early-80s
    (1983, 1989): 0.035,  # 3.5% late-80s
    (1990, 1999): 0.025,  # 2.5% 1990s
    (2000, 2019): 0.020,  # 2.0% 2000s-2010s
}

adjusted_historical = adjust_returns_for_dividends(sp500, annual_div_yield=div_schedule)

print("Time-varying dividend adjustment:")
print(f"Raw avg daily return: {sp500['Raw Change (bps)'].mean():.2f} bps")
print(f"Total avg daily return: {adjusted_historical['Total Return (bps)'].mean():.2f} bps")

# Show some weekend examples
print("\nNotice 3x dividend on weekends (Days Since Prev = 3):")
adjusted_historical[['Date', 'Days Since Prev', 'Raw Change (bps)', 'Daily Div Yield (bps)', 'Total Return (bps)']].head(10)

Time-varying dividend adjustment:
Raw avg daily return: 3.88 bps
Total avg daily return: 4.88 bps

Notice 3x dividend on weekends (Days Since Prev = 3):


,Date,Days Since Prev,Raw Change (bps),Daily Div Yield (bps),Total Return (bps)
0,1979-12-26,0,9.0,1.206015,10.197797
1,1979-12-27,1,19.0,1.206015,20.197797
2,1979-12-28,1,-19.0,1.206015,-17.802203
3,1979-12-31,3,9.0,3.618481,12.593827
4,1980-01-01,1,0.0,1.206015,1.197797
5,1980-01-02,1,-195.0,1.206015,-193.802203
6,1980-01-03,1,-57.0,1.206015,-55.802203
7,1980-01-04,1,124.0,1.206015,125.197797
8,1980-01-07,3,28.0,3.618481,31.593827
9,1980-01-08,1,197.0,1.206015,198.197797


## Calculate Leveraged Returns

Now let's see how leverage affects returns after accounting for borrowing costs (RFR + 1% margin spread).

In [17]:
# Calculate leveraged returns with 2x leverage
leveraged_2x = calculate_leveraged_returns(adjusted_simple, fedfunds, leverage=2.0)

print("Leverage Comparison:")
print(f"1x (no leverage): {adjusted_simple['Total Return (bps)'].mean():.2f} bps/day")
print(f"2x leverage: {leveraged_2x['Leveraged Return (bps)'].mean():.2f} bps/day")
print(f"\nAvg borrowing cost: {leveraged_2x['Daily Leverage Cost (bps)'].mean():.2f} bps/day")
print(f"Avg RFR: {leveraged_2x['Daily RFR (bps)'].mean():.2f} bps/day")

# Show weekend compounding of borrowing costs
print("\nNotice borrowing costs also compound on weekends:")
leveraged_2x[['Date', 'Days Since Prev', 'Total Return (bps)', 'Daily Leverage Cost (bps)', 'Leveraged Return (bps)']].head(10)

NameError: name 'adjusted_simple' is not defined